In [15]:
import pandas as pd
import datetime as datetime

In [45]:
def parse_date_flex(date_str):
    if pd.isna(date_str):
        return pd.NaT
    
    date_str = str(date_str).strip()
    possible_formats = [
        "%Y-%m-%d",
        "%Y/%m/%d",
        "%d/%m/%Y",
        "%d-%m-%Y",
        "%m/%d/%Y",
        "%m-%d-%Y"
    ]

    for fmt in possible_formats:
        try:
            return datetime.datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    return pd.NaT
    

In [56]:
fb = pd.read_csv('../data/fb_ads.csv')
google = pd.read_csv('../data/google_ads.csv')
tiktok = pd.read_csv('../data/tiktok_ads.csv')

In [57]:
fb.head()

,date,campaign_name,spend_usd,impressions
0,2025-10-01,Autumn Sale,25.5,12000
1,2025/10/02,Winter Deals,40.2,18000
2,10-03-2025,New Product Launch,32.0,25000
3,2025-10-04,Retargeting Campaign,15.8,9000
4,03/10/2025,Brand Awareness,28.1,15000


In [58]:
fb.loc[fb['campaign_name'] == "New Product Launch", 'date'] = "2025-10-03"

In [59]:
fb['platform'] = 'facebook'
fb['date'] = fb['date'].apply(parse_date_flex)
fb['biaya_idr'] = fb['spend_usd'] * 16000
fb.rename(columns={
    'campaign_name': 'nama_kampanye',
    'impressions': 'metric_1'
}, inplace=True)
fb['metric_2'] = None
fb = fb[['date', 'platform', 'nama_kampanye', 'biaya_idr', 'metric_1', 'metric_2']]


In [60]:
fb.head()

,date,platform,nama_kampanye,biaya_idr,metric_1,metric_2
0,2025-10-01,facebook,Autumn Sale,408000.0,12000,None
1,2025-10-02,facebook,Winter Deals,643200.0,18000,None
2,2025-10-03,facebook,New Product Launch,512000.0,25000,None
3,2025-10-04,facebook,Retargeting Campaign,252800.0,9000,None
4,2025-10-03,facebook,Brand Awareness,449600.0,15000,None


In [61]:
google.head()

,day,campaign_id,cost_idr,clicks
0,2025-10-01,GA_001,125000,450
1,02/10/2025,GA_002,98000,320
2,2025/10/03,GA_003,143500,510
3,04-10-2025,GA_004,75000,280
4,05-10-2025,GA_005,165000,620


In [62]:
google['platform'] = 'google'
google['day'] = google['day'].apply(parse_date_flex)
google.rename(columns={
    'day': 'date',
    'campaign_id': 'nama_kampanye',
    'cost_idr': 'biaya_idr',
    'clicks': 'metric_1'
}, inplace=True)
google['metric_2'] = None
google = google[['date', 'platform', 'nama_kampanye', 'biaya_idr', 'metric_1', 'metric_2']]

In [63]:
google.head()

,date,platform,nama_kampanye,biaya_idr,metric_1,metric_2
0,2025-10-01,google,GA_001,125000,450,None
1,2025-10-02,google,GA_002,98000,320,None
2,2025-10-03,google,GA_003,143500,510,None
3,2025-10-04,google,GA_004,75000,280,None
4,2025-10-05,google,GA_005,165000,620,None


In [64]:
tiktok.head()

,tanggal,nama_iklan,biaya
0,01/10/2025,Promo Diskon Besar,Rp 150.000
1,2025-10-02,Video UGC 1,Rp 200.500
2,03-10-2025,Kampanye Challenge,Rp 175.000
3,2025/10/04,Iklan Awareness,Rp 90.000
4,05/10/2025,Testimoni Pelanggan,Rp 220.000


In [65]:
tiktok['platform'] = 'tiktok'
tiktok['tanggal'] = tiktok['tanggal'].apply(parse_date_flex)
tiktok['biaya'] = (
    tiktok['biaya'].astype(str)
    .str.replace('Rp', '', regex=False)
    .str.replace('.', '', regex=False)
    .str.strip().astype(float)
)
tiktok.rename(columns={
    'tanggal': 'date',
    'nama_iklan': 'nama_kampanye',
    'biaya': 'biaya_idr'
}, inplace=True)
tiktok['metric_1'] = None
tiktok['metric_2'] = None
tiktok = tiktok[['date', 'platform', 'nama_kampanye', 'biaya_idr', 'metric_1', 'metric_2']]


In [66]:
tiktok.head()

,date,platform,nama_kampanye,biaya_idr,metric_1,metric_2
0,2025-10-01,tiktok,Promo Diskon Besar,150000.0,None,None
1,2025-10-02,tiktok,Video UGC 1,200500.0,None,None
2,2025-10-03,tiktok,Kampanye Challenge,175000.0,None,None
3,2025-10-04,tiktok,Iklan Awareness,90000.0,None,None
4,2025-10-05,tiktok,Testimoni Pelanggan,220000.0,None,None


In [67]:
merged_df = pd.concat([fb, google, tiktok], ignore_index=True)
merged_df = merged_df.sort_values(by='date').reset_index(drop=True)

In [68]:
merged_df

,date,platform,nama_kampanye,biaya_idr,metric_1,metric_2
0,2025-10-01,facebook,Autumn Sale,408000.0,12000,None
1,2025-10-01,google,GA_001,125000.0,450,None
2,2025-10-01,tiktok,Promo Diskon Besar,150000.0,None,None
3,2025-10-02,facebook,Winter Deals,643200.0,18000,None
4,2025-10-02,google,GA_002,98000.0,320,None
5,2025-10-02,tiktok,Video UGC 1,200500.0,None,None
6,2025-10-03,facebook,New Product Launch,512000.0,25000,None
7,2025-10-03,facebook,Brand Awareness,449600.0,15000,None
8,2025-10-03,google,GA_003,143500.0,510,None
9,2025-10-03,tiktok,Kampanye Challenge,175000.0,None,None


In [69]:
merged_df.to_csv('../data/cleaned_ads_v2.csv', index=False)